In [2]:
import json
import pandas as pd
import requests
from sqlalchemy import create_engine
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\Get_Data_Functions.py

In [4]:
def list_db_tables(db_name):
    Conn = db.connect(db_name)
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor = Conn.cursor()
    cursor.execute(sql_query)
    print(cursor.fetchall())
    Conn.close() 

In [5]:
list_db_tables('MacroData.db')

[]


In [7]:
def create_year_increments(startyear, endyear):
    """
    Create start and endyear increments of 20 years or 
    """
    if endyear - startyear < 20:
        return startyear, endyear
    else:
        i = startyear
        increments = []
        while i <= endyear: 
            start_end_array = []
            start_end_array.append(i)
            if i + 19 > endyear:
                start_end_array.append(endyear)
            else:
                start_end_array.append(i + 19)
            increments.append(start_end_array)
            i += 20
    
    return increments

In [8]:
increments = create_year_increments(1972,2024)
i = 0
df_data = pd.DataFrame() 
df_dict = pd.DataFrame() 
while i < len(increments):
    #print(increments[i][0])
    
    message, json_data = get_BLS_data(["CUSR0000SA0"], increments[i][0], increments[i][1])
    if not df_data.empty:
        df_data = pd.concat([df_data, pd.DataFrame(json_data['Results']['series'][0]['data'])])
    if df_data.empty:
        df_data = pd.DataFrame(json_data['Results']['series'][0]['data'])
    if df_dict.empty: 
        df_dict = create_dict_dataframe(json_data)
    i += 1

df_data['month'] = df_data['year'] + df_data['period'].str.replace('M','')
df_data['month'] = pd.to_datetime(df_data['month'],format='%Y%m')
df_data['month'] = df_data['month'].dt.date.apply(lambda x: x.strftime('%Y-%m'))
df_data = df_data.sort_values(by=['month'])
df_data['pct_changeMoM'] = df_data['value'].astype(float).pct_change() * 100
df_data['pct_changeYoY'] = df_data['value'].astype(float).pct_change(12) * 100
df_data['pct_changeMoM_Ann'] = ((df_data['value'].astype(float).pct_change() + 1) ** 12 - 1) * 100
df_data = df_data.drop(columns=['year', 'period','periodName', 'latest','footnotes'])
df_data  

,value,month,pct_changeMoM,pct_changeYoY,pct_changeMoM_Ann
239,41.20,1972-01,NaN,NaN,NaN
238,41.40,1972-02,0.485437,NaN,5.983315
237,41.40,1972-03,0.000000,NaN,0.000000
236,41.50,1972-04,0.241546,NaN,2.937370
235,41.60,1972-05,0.240964,NaN,2.930198
...,...,...,...,...,...
4,312.230,2024-03,0.378069,3.475131,4.632370
3,313.207,2024-04,0.312910,3.357731,3.820226
2,313.225,2024-05,0.005747,3.250210,0.068986
1,313.049,2024-06,-0.056190,2.975629,-0.672196


In [9]:
df_dict

,series_title,seasonality,survey_long_name,survey_short_name,survey_abbreviation,measure_data_type,area,item
series_id,,,,,,,,
CUSR0000SA0,"All items in U.S. city average, all urban cons...",Seasonally Adjusted,Consumer Price Index for All Urban Consumers (...,CPI-U,CU,All items,U.S. city average,All items


In [10]:
df_data.to_csv('CPI.csv', index=False)

In [11]:
conn = db.connect('MacroData.db')
df_data.to_sql(df_dict.index[0] + '_data', conn, if_exists='replace')

631

In [3]:
# create connection to the database
connection = db.connect('MacroData.db') 
  
# drop table 
connection.execute("DROP TABLE CUSR0000SA0_data") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully
